In [1]:
import numpy as np 
from sklearn.utils.class_weight import compute_class_weight

In [2]:
import pandas as pd
financial_dataset = pd.read_csv('latestpull.csv')
nonfinancial_dataset = pd.read_csv('nonfinancialdata.csv')

financial_dataset.drop_duplicates(inplace=True)
nonfinancial_dataset.drop_duplicates(inplace=True)

to_test1 = pd.read_csv('companyscrape.csv')
to_test2 = pd.read_csv('companyscrape2.csv')
to_test3 = pd.read_csv('companyscrape4.csv')

to_test1.drop_duplicates(inplace=True)
to_test2.drop_duplicates(inplace=True)
to_test3.drop_duplicates(inplace=True)

to_test = pd.concat([to_test1,to_test2,to_test3])

to_test.drop_duplicates(inplace=True)

In [ ]:
financial_dataset['label']=1
nonfinancial_dataset['label']=0

In [7]:
financial_dataset_20 = financial_dataset.head(20000)

In [8]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
both_datasets = pd.concat([financial_dataset_20, nonfinancial_dataset])
both_datasets['Content'].fillna('', inplace=True)

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    both_datasets['Content'], both_datasets['label'], test_size=0.1, random_state=42
)

# Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('Untitled Folder 1')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

# Convert to tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': train_encodings['input_ids'], 
        'attention_mask': train_encodings['attention_mask']
    }, 
    train_labels
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': val_encodings['input_ids'], 
        'attention_mask': val_encodings['attention_mask']
    }, 
    val_labels
)).batch(16)

# Load the TensorFlow model
model = TFDistilBertForSequenceClassification.from_pretrained('Untitled Folder 1', num_labels=2)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = dict(enumerate(class_weights))

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
# Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1)

# Fit the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2,
    class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr]
)


2024-04-07 17:11:12.216053: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 17:11:12.216145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 17:11:12.234494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 17:11:12.291552: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/nleffler/.conda/envs/tf2-gpu/

Epoch 1/3


2024-04-07 17:14:13.424604: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-07 17:14:15.279769: I external/local_xla/xla/service/service.cc:168] XLA service 0x151c7b7b12a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-07 17:14:15.279801: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-04-07 17:14:15.295652: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-07 17:14:15.332200: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1712524455.398613 2343413 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1424/1424 [==============================] - 304s 203ms/step - loss: 1.5529 - accuracy: 0.3527 - val_loss: 0.3510 - val_accuracy: 0.1743 - lr: 5.0000e-05
Epoch 2/3
1424/1424 [==============================] - 287s 201ms/step - loss: 0.3399 - accuracy: 0.6825 - val_loss: 0.3283 - val_accuracy: 0.8289 - lr: 5.0000e-05
Epoch 3/3
1424/1424 [==============================] - 287s 201ms/step - loss: 1.8712 - accuracy: 0.7483 - val_loss: 0.4138 - val_accuracy: 0.1842 - lr: 5.0000e-05


In [13]:
to_test['Content'] = to_test['Content'].fillna('')
test_texts = to_test['Content'].tolist()  # Make sure to replace 'Content' with the name of the column containing texts
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': test_encodings['input_ids'], 
        'attention_mask': test_encodings['attention_mask']
    }
)).batch(16)  

# Make predictions
predictions = model.predict(test_dataset)
logits = predictions.logits

# Convert logits to class labels
# For binary classification:
predicted_labels = tf.sigmoid(logits).numpy().round().astype(int)


15893/15893 [==============================] - 1012s 64ms/step


ValueError: Expected a 1D array, got an array with shape (254276, 2)

In [35]:
probabilities = tf.sigmoid(logits).numpy()

# Set a new threshold
threshold = 0.7  # for example

predicted_labels = (probabilities > threshold).astype(int)

In [36]:
to_test['Predicted_Label'] = predicted_labels

In [37]:
to_test['Keyword'].value_counts()

Keyword
Avery Dennison             11002
American Electric Power    11000
Allegro MicroSystems       11000
American Financial         11000
Antero Midstream           11000
                           ...  
Maravai LifeSciences          70
Liberty Live                  57
Rithm Capital                 49
NCR Voyix                     11
NCR Atleos                     4
Name: count, Length: 102, dtype: int64

In [38]:
to_test['Predicted_Label'].value_counts()

Predicted_Label
0    221273
1     33003
Name: count, dtype: int64

In [39]:
to_test[to_test['Content'].str.contains('stock', na=False)]['Predicted_Label'].value_counts()

Predicted_Label
1    18685
0     1680
Name: count, dtype: int64

In [42]:
to_test.to_csv('LabeledDistillation.csv')

In [41]:
to_test.head(50)

,Unnamed: 0,Date,Keyword,Title,Subreddit,Content,Predicted_Label
0,0,2023-02-20,Booz Allen Hamilton,Booz Allen Hamilton sued for ripping off Natio...,NationalPark,While many think the Recreation.gov website be...,0
1,1,2023-12-01,Booz Allen Hamilton,Booz Allen Hamilton’s insulting recruiting pra...,recruiting,Update 12.6.2023 - and to rub this in my face ...,0
2,2,2024-02-14,Booz Allen Hamilton,Question about job at Booz Allen Hamilton,nova,Does anyone here work at BAH? I applied for a ...,0
3,3,2022-12-11,Booz Allen Hamilton,Your friendly neighborhood Booz Allen Hamilton...,WildernessBackpacking,,0
4,4,2024-03-11,Booz Allen Hamilton,Booz Allen Hamilton - Advice Needed,ITCareerQuestions,Looking for some career negotiation help on an...,0
5,5,2024-02-15,Booz Allen Hamilton,Booz Allen Hamilton After Graduation,jobs,"Hi All,\n\nI’m a senior majoring in health ser...",0
6,6,2023-12-12,Booz Allen Hamilton,Booz Allen Hamilton - Associate - Salary advice?,careerguidance,I've worked with Booz Allen Hamilton for almos...,0
7,7,2024-02-15,Booz Allen Hamilton,Booz Allen Hamilton Artificial Intelligence Jo...,nova,Does anyone here work in AI at BAH? I recently...,0
8,8,2023-07-22,Booz Allen Hamilton,Booz Allen Hamilton to pay $377 million for fa...,politics,,0
9,9,2022-11-30,Booz Allen Hamilton,Why Is Booz Allen Renting Us Back Our Own Nati...,WildernessBackpacking,,0
